In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
import json

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

## Data preprocessing

In [2]:
print("Downloading movielens data...")
from urllib.request import urlretrieve
import zipfile

# Download the data
urlretrieve("http://files.grouplens.org/datasets/movielens/ml-20m.zip", "movielens.zip")
zip_ref = zipfile.ZipFile('movielens.zip', "r")
zip_ref.extractall()


In [3]:
# load in the data
df = pd.read_csv('ml-20m/ratings.csv')

In [4]:
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [6]:
bs = 128
reg = 0. # regularization penalty


In [7]:
# Customize dataset
N = df.userId.max() + 1 # number of users
M = df.movieId.max() + 1 # number of movies

# split into train and test
df = shuffle(df)
df = df.iloc[:100000]
cutoff = int(0.8*len(df))
cutoffeval = int(0.9*len(df))
df_train = df.iloc[:cutoff]
df_eval = df.iloc[cutoff:cutoffeval]
df_test = df.iloc[cutoffeval:]

# initialize variables
K = 10 # latent dimensionality
mu = df_train.rating.mean()


In [8]:
N,M,K,len(df_train),len(df), mu

(138494, 131263, 10, 80000, 100000, 3.52466875)

In [9]:
class ratingdataset(Dataset):
  def __init__(self, df):
    self.df = df
    self.u = df.userId.values
    self.m = df.movieId.values
    self.r = df.rating.values
    self.N = self.u.max() + 1
    self.M = self.m.max() + 1

  def __len__(self):
    return len(self.u)
  
  def __getitem__(self, idx):
    return [self.u[idx], self.m[idx]], self.r[idx]


In [10]:
ratingData_train = ratingdataset(df_train)
ratingData_eval = ratingdataset(df_eval)
ratingData_test = ratingdataset(df_test)

In [11]:
train_dataloader = DataLoader(ratingData_train, batch_size=bs, shuffle=True)
eval_dataloader = DataLoader(ratingData_eval, batch_size=bs, shuffle=True)
test_dataloader = DataLoader(ratingData_test, batch_size=bs, shuffle=True)

In [12]:
train_dataloader, eval_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7ff45b890c50>,
 <torch.utils.data.dataloader.DataLoader at 0x7ff45b8b2950>)

In [13]:
# # direct data
# users = torch.tensor(df_train.userId.values)
# items = torch.tensor(df_train.movieId.values)
# ratings = torch.tensor(df_train.rating.values)

In [71]:
class UserItemEmbeddingNet(torch.nn.Module):
    def __init__(self, n_users, n_items, k_factors, embedding_dropout = 0.02, hidden = 10, dropouts = 0.2):

        super(UserItemEmbeddingNet, self).__init__()
        self.u = torch.nn.Embedding(n_users, k_factors)
        self.m = torch.nn.Embedding(n_items, k_factors)
        self.u_bias = torch.nn.Embedding(n_users, 1)
        self.m_bias = torch.nn.Embedding(n_items, 1)

    def forward(self, users, items):
        uembed = self.u(users)
        membed = self.m(items)
        ubias = self.u_bias(users)
        mbias = self.m_bias(items)
        umdot = torch.mul(uembed,membed)
        umdot = torch.sum(umdot,1)
        umdot = torch.reshape(umdot, (umdot.shape[0], 1))
        output = torch.add(umdot, ubias)
        output = torch.add(output, mbias)
        output = torch.flatten(output)
        return output

UserItemEmbeddingNet = UserItemEmbeddingNet(N, M, K)
print(UserItemEmbeddingNet)

UserItemEmbeddingNet(
  (u): Embedding(138494, 10)
  (m): Embedding(131263, 10)
  (u_bias): Embedding(138494, 1)
  (m_bias): Embedding(131263, 1)
)


In [72]:
epochs = 100
lr = 0.1
min_valid_loss = np.inf

def train_loop(epoch):
    train_loss = 0.0
    UserItemEmbeddingNet.train()
    loss_func = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(UserItemEmbeddingNet.parameters(), lr=lr, momentum=0.9)
    
    for i, data in tqdm(enumerate(train_dataloader)):
        inputs, labels = data
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()
        users, items = inputs[0], inputs[1]

        optimizer.zero_grad()
        target = UserItemEmbeddingNet(users, items)
        loss = loss_func(target.float(), labels.float())
        #loss = loss_func(target, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    valid_loss = 0.0
    UserItemEmbeddingNet.eval()     # Optional when not using model Specific layer
    for data in eval_dataloader:
        inputs, labels = data
        if torch.cuda.is_available():
            inputs, labels =inputs.cuda(), labels.cuda()
        users, items = inputs[0], inputs[1]
        target = UserItemEmbeddingNet(users, items)
        loss = loss_func(target.float(), labels.float())
        valid_loss = loss.item() * len(inputs)

    if epoch%1==0:
        print(f'Epoch {epoch+1} \t\t Training Loss: {train_loss / len(train_dataloader)} \t\t Validation Loss: {valid_loss / len(eval_dataloader)}')
    if min_valid_loss > valid_loss:
        print(f"Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f})")
        min_valid_loss = valid_loss
        # Saving State Dict
        # torch.save(UserItemEmbeddingNet.state_dict(), 'saved_UserItemEmbeddingNet.pth')
 

In [73]:
for epoch in range(epochs):
    train_loop(epoch)

625it [00:09, 66.11it/s]


Epoch 1 		 Training Loss: 16.278217889404296 		 Validation Loss: 0.1296306320383579
Validation Loss Decreased(inf--->10.240820)


625it [00:09, 67.92it/s]


Epoch 2 		 Training Loss: 5.1274379463195805 		 Validation Loss: 0.13241595256177685
Validation Loss Decreased(inf--->10.460860)


625it [00:09, 68.73it/s]


...

Epoch 98 		 Training Loss: 0.01916594110429287 		 Validation Loss: 0.3899898046179663
Validation Loss Decreased(inf--->30.809195)


625it [00:09, 67.90it/s]


Epoch 99 		 Training Loss: 0.018682767954096197 		 Validation Loss: 0.3292795012268839
Validation Loss Decreased(inf--->26.013081)


625it [00:08, 69.51it/s]

Epoch 100 		 Training Loss: 0.01819442109167576 		 Validation Loss: 0.1079769617394556
Validation Loss Decreased(inf--->8.530180)


In [74]:
tusers = torch.tensor(df_test.userId.values)
titems = torch.tensor(df_test.movieId.values)
tratings = df_test.rating.values

In [75]:
with torch.no_grad():
    tpreds = UserItemEmbeddingNet(tusers, titems)

In [77]:
tpreds, len(tpreds)

(tensor([6.2959, 4.0425, 4.4953,  ..., 5.7096, 3.1354, 5.3332]), 10000)

In [78]:
tpredsfinal = tpreds.detach().numpy()
#tpredsfinal = (tpredsfinal/0.5).astype(int)
tpredsfinal = (tpredsfinal>3).astype(int)

In [79]:
#tratings = (tratings*2).astype(int)
tratings = (tratings>3).astype(int)
tratings, tpredsfinal

(array([0, 0, 1, ..., 0, 1, 0]), array([1, 1, 1, ..., 1, 1, 1]))

In [80]:
from sklearn.metrics import f1_score
f1_score(tpredsfinal, tratings)

0.5923937360178971